# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import requests
import json
from datetime import datetime
from dotenv import find_dotenv, load_dotenv

# Preparing Bike Trip Data

In [2]:
# Listing trip data files
Path = r'D:\Data_Analysis\05-12-2025_Bike_Dashboard\02.Data'
files_folder = os.path.join(Path, 'Original Data')
file_list = [os.path.join(files_folder, name) for name in os.listdir(files_folder)]
file_list

['D:\\Data_Analysis\\05-12-2025_Bike_Dashboard\\02.Data\\Original Data\\JC-202201-citibike-tripdata.csv',
 'D:\\Data_Analysis\\05-12-2025_Bike_Dashboard\\02.Data\\Original Data\\JC-202202-citibike-tripdata.csv',
 'D:\\Data_Analysis\\05-12-2025_Bike_Dashboard\\02.Data\\Original Data\\JC-202203-citibike-tripdata.csv',
 'D:\\Data_Analysis\\05-12-2025_Bike_Dashboard\\02.Data\\Original Data\\JC-202204-citibike-tripdata.csv',
 'D:\\Data_Analysis\\05-12-2025_Bike_Dashboard\\02.Data\\Original Data\\JC-202205-citibike-tripdata.csv',
 'D:\\Data_Analysis\\05-12-2025_Bike_Dashboard\\02.Data\\Original Data\\JC-202206-citibike-tripdata.csv',
 'D:\\Data_Analysis\\05-12-2025_Bike_Dashboard\\02.Data\\Original Data\\JC-202207-citbike-tripdata.csv',
 'D:\\Data_Analysis\\05-12-2025_Bike_Dashboard\\02.Data\\Original Data\\JC-202208-citibike-tripdata.csv',
 'D:\\Data_Analysis\\05-12-2025_Bike_Dashboard\\02.Data\\Original Data\\JC-202209-citibike-tripdata.csv',
 'D:\\Data_Analysis\\05-12-2025_Bike_Dashboard\

In [3]:
# Concatenating all CSV files into a data frame
df_trips = pd.concat((pd.read_csv(f) for f in file_list), ignore_index=True)
df_trips.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,CA5837152804D4B5,electric_bike,2022-01-26 18:50:39,2022-01-26 18:51:53,12 St & Sinatra Dr N,HB201,12 St & Sinatra Dr N,HB201,40.750604,-74.024020,40.750604,-74.024020,member
1,BA06A5E45B6601D2,classic_bike,2022-01-28 13:14:07,2022-01-28 13:20:23,Essex Light Rail,JC038,Essex Light Rail,JC038,40.712774,-74.036486,40.712774,-74.036486,member
2,7B6827D7B9508D93,classic_bike,2022-01-10 19:55:13,2022-01-10 20:00:37,Essex Light Rail,JC038,Essex Light Rail,JC038,40.712774,-74.036486,40.712774,-74.036486,member
3,6E5864EA6FCEC90D,electric_bike,2022-01-26 07:54:57,2022-01-26 07:55:22,12 St & Sinatra Dr N,HB201,12 St & Sinatra Dr N,HB201,40.750604,-74.024020,40.750604,-74.024020,member
4,E24954255BBDE32D,electric_bike,2022-01-13 18:44:46,2022-01-13 18:45:43,12 St & Sinatra Dr N,HB201,12 St & Sinatra Dr N,HB201,40.750604,-74.024020,40.750604,-74.024020,member


The df_trips has been created by concatenating the monthly trip data with a generator comprehension after reading all 12 CSV files whose paths were listed by the list comprehension that was in charge of gathering all names of files and appending these to the files_folder path.

In [4]:
df_trips.shape

(895485, 13)

In [5]:
df_trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 895485 entries, 0 to 895484
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             895485 non-null  object 
 1   rideable_type       895485 non-null  object 
 2   started_at          895485 non-null  object 
 3   ended_at            895485 non-null  object 
 4   start_station_name  895475 non-null  object 
 5   start_station_id    895475 non-null  object 
 6   end_station_name    892281 non-null  object 
 7   end_station_id      892281 non-null  object 
 8   start_lat           895485 non-null  float64
 9   start_lng           895485 non-null  float64
 10  end_lat             893515 non-null  float64
 11  end_lng             893515 non-null  float64
 12  member_casual       895485 non-null  object 
dtypes: float64(4), object(9)
memory usage: 88.8+ MB


## Deriving Date Column for Merging

In [6]:
# Converting 'started_at' from object to datetime format
df_trips['started_at'] = pd.to_datetime(df_trips['started_at'], format='%Y-%m-%d %H:%M:%S')

# Creating date column in YYYY-MM-DD string format from started_at
df_trips['date'] = pd.to_datetime(df_trips['started_at'], format='%Y-%m-%d').dt.date

# Converting date object to datetime format
df_trips['date'] = pd.to_datetime(df_trips['date'])

df_trips

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date
0,CA5837152804D4B5,electric_bike,2022-01-26 18:50:39,2022-01-26 18:51:53,12 St & Sinatra Dr N,HB201,12 St & Sinatra Dr N,HB201,40.750604,-74.024020,40.750604,-74.024020,member,2022-01-26
1,BA06A5E45B6601D2,classic_bike,2022-01-28 13:14:07,2022-01-28 13:20:23,Essex Light Rail,JC038,Essex Light Rail,JC038,40.712774,-74.036486,40.712774,-74.036486,member,2022-01-28
2,7B6827D7B9508D93,classic_bike,2022-01-10 19:55:13,2022-01-10 20:00:37,Essex Light Rail,JC038,Essex Light Rail,JC038,40.712774,-74.036486,40.712774,-74.036486,member,2022-01-10
3,6E5864EA6FCEC90D,electric_bike,2022-01-26 07:54:57,2022-01-26 07:55:22,12 St & Sinatra Dr N,HB201,12 St & Sinatra Dr N,HB201,40.750604,-74.024020,40.750604,-74.024020,member,2022-01-26
4,E24954255BBDE32D,electric_bike,2022-01-13 18:44:46,2022-01-13 18:45:43,12 St & Sinatra Dr N,HB201,12 St & Sinatra Dr N,HB201,40.750604,-74.024020,40.750604,-74.024020,member,2022-01-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895480,D438F1622839AC50,classic_bike,2022-12-06 15:43:38,2022-12-06 15:53:57,Dey St,JC065,Riverview Park,JC057,40.737828,-74.067083,40.744319,-74.043991,member,2022-12-06
895481,747A63A8E782D171,electric_bike,2022-12-08 08:17:51,2022-12-08 08:23:33,9 St HBLR - Jackson St & 8 St,HB305,City Hall - Washington St & 1 St,HB105,40.747907,-74.038412,40.737360,-74.030970,casual,2022-12-08
895482,AE090858CFDE6E82,electric_bike,2022-12-23 14:10:07,2022-12-23 14:14:18,Mama Johnson Field - 4 St & Jackson St,HB404,City Hall - Washington St & 1 St,HB105,40.743140,-74.040041,40.737360,-74.030970,member,2022-12-23
895483,B3CC8E70AF4E259C,classic_bike,2022-12-02 04:43:25,2022-12-02 04:46:55,Mama Johnson Field - 4 St & Jackson St,HB404,City Hall - Washington St & 1 St,HB105,40.743135,-74.040080,40.737360,-74.030970,member,2022-12-02


In [7]:
df_trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 895485 entries, 0 to 895484
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   ride_id             895485 non-null  object        
 1   rideable_type       895485 non-null  object        
 2   started_at          895485 non-null  datetime64[ns]
 3   ended_at            895485 non-null  object        
 4   start_station_name  895475 non-null  object        
 5   start_station_id    895475 non-null  object        
 6   end_station_name    892281 non-null  object        
 7   end_station_id      892281 non-null  object        
 8   start_lat           895485 non-null  float64       
 9   start_lng           895485 non-null  float64       
 10  end_lat             893515 non-null  float64       
 11  end_lng             893515 non-null  float64       
 12  member_casual       895485 non-null  object        
 13  date                895485 no

## Creating Station Summary Statistics

In [8]:
# Creating summary statistics per station
startMem = df_trips['member_casual'].value_counts('start_station_id')
endMem = df_trips['member_casual'].value_counts('end_station_id')
startTotal = df_trips['start_station_id'].nunique()
endTotal = df_trips['end_station_id'].nunique()
startCount = df_trips['start_station_id'].value_counts()
endCount = df_trips['end_station_id'].value_counts()

print('START STATIONS TOTAL: ', startTotal, '\n', f'Top 75 Account for: {startCount.head(
    75).sum()/len(df_trips):.2%} of total trips', '\n', 'Member Percentage based on ', startMem)

print('\n\n', 'END STATIONS TOTAL: ', endTotal, '\n', f'Top 75 Account for: {endCount.head(
    75).sum()/len(df_trips):.2%} of total trips', '\n', 'Member Percentage based on ', endMem)

START STATIONS TOTAL:  86 
 Top 75 Account for: 98.55% of total trips 
 Member Percentage based on  member_casual
member    0.657532
casual    0.342468
Name: proportion, dtype: float64


 END STATIONS TOTAL:  319 
 Top 75 Account for: 98.10% of total trips 
 Member Percentage based on  member_casual
member    0.657532
casual    0.342468
Name: proportion, dtype: float64


The dataset is comprised of 86 stations where bikes are rented out and 319 stations where bikes are returned. End station selection seem to be aligned according to the start stations as both account for about 98% of trips and are comprised of the same proportion of member type. There seems to be too many end stations with an insignificant amount of activity.

# Sourcing Weather Data using NOAA's API

In [9]:
# Finding and loading environment variables
dotenvPath = find_dotenv()
load_dotenv(dotenvPath)
Token = os.getenv('NOAA_TOKEN')

In [10]:
# Getting La Guardia, NY API data of 2022
raw = requests.get(
    'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00014732&startdate=2022-01-01&enddate=2022-12-31', headers={'token': Token})
raw_data = json.loads(raw.text)
raw_data

{'metadata': {'resultset': {'offset': 1, 'count': 365, 'limit': 1000}},
 'results': [{'date': '2022-01-01T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 116},
  {'date': '2022-01-02T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 114},
  {'date': '2022-01-03T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 14},
  {'date': '2022-01-04T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': -27},
  {'date': '2022-01-05T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 32},
  {'date': '2022-01-06T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 49},
  {'date': '2022-01-07T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attribut

## Saving Raw Weather Data

In [11]:
# Exporting raw_data as a data frame
weather = raw_data['results']
df_weather = pd.DataFrame(weather)
df_weather.to_csv(os.path.join(Path, 'Prepared Data', 'La_Guardia_Weather.csv'), index=False)

## Creating a Temperatures Data Frame

In [12]:
# Getting all items in the response that correspond to TAVG
avg_temps = [item for item in raw_data['results'] if item['datatype'] == 'TAVG']
avg_temps

[{'date': '2022-01-01T00:00:00',
  'datatype': 'TAVG',
  'station': 'GHCND:USW00014732',
  'attributes': 'H,,S,',
  'value': 116},
 {'date': '2022-01-02T00:00:00',
  'datatype': 'TAVG',
  'station': 'GHCND:USW00014732',
  'attributes': 'H,,S,',
  'value': 114},
 {'date': '2022-01-03T00:00:00',
  'datatype': 'TAVG',
  'station': 'GHCND:USW00014732',
  'attributes': 'H,,S,',
  'value': 14},
 {'date': '2022-01-04T00:00:00',
  'datatype': 'TAVG',
  'station': 'GHCND:USW00014732',
  'attributes': 'H,,S,',
  'value': -27},
 {'date': '2022-01-05T00:00:00',
  'datatype': 'TAVG',
  'station': 'GHCND:USW00014732',
  'attributes': 'H,,S,',
  'value': 32},
 {'date': '2022-01-06T00:00:00',
  'datatype': 'TAVG',
  'station': 'GHCND:USW00014732',
  'attributes': 'H,,S,',
  'value': 49},
 {'date': '2022-01-07T00:00:00',
  'datatype': 'TAVG',
  'station': 'GHCND:USW00014732',
  'attributes': 'H,,S,',
  'value': 7},
 {'date': '2022-01-08T00:00:00',
  'datatype': 'TAVG',
  'station': 'GHCND:USW00014732',

In [13]:
# Getting only date field from all average temperature readings
dates_temp = [item['date'] for item in avg_temps]
dates_temp

['2022-01-01T00:00:00',
 '2022-01-02T00:00:00',
 '2022-01-03T00:00:00',
 '2022-01-04T00:00:00',
 '2022-01-05T00:00:00',
 '2022-01-06T00:00:00',
 '2022-01-07T00:00:00',
 '2022-01-08T00:00:00',
 '2022-01-09T00:00:00',
 '2022-01-10T00:00:00',
 '2022-01-11T00:00:00',
 '2022-01-12T00:00:00',
 '2022-01-13T00:00:00',
 '2022-01-14T00:00:00',
 '2022-01-15T00:00:00',
 '2022-01-16T00:00:00',
 '2022-01-17T00:00:00',
 '2022-01-18T00:00:00',
 '2022-01-19T00:00:00',
 '2022-01-20T00:00:00',
 '2022-01-21T00:00:00',
 '2022-01-22T00:00:00',
 '2022-01-23T00:00:00',
 '2022-01-24T00:00:00',
 '2022-01-25T00:00:00',
 '2022-01-26T00:00:00',
 '2022-01-27T00:00:00',
 '2022-01-28T00:00:00',
 '2022-01-29T00:00:00',
 '2022-01-30T00:00:00',
 '2022-01-31T00:00:00',
 '2022-02-01T00:00:00',
 '2022-02-02T00:00:00',
 '2022-02-03T00:00:00',
 '2022-02-04T00:00:00',
 '2022-02-05T00:00:00',
 '2022-02-06T00:00:00',
 '2022-02-07T00:00:00',
 '2022-02-08T00:00:00',
 '2022-02-09T00:00:00',
 '2022-02-10T00:00:00',
 '2022-02-11T00:

In [14]:
# Getting the temperature from all average temperature readings
temps = [item['value'] for item in avg_temps]
temps

[116,
 114,
 14,
 -27,
 32,
 49,
 7,
 -25,
 14,
 16,
 -54,
 -19,
 40,
 48,
 -67,
 -80,
 39,
 18,
 32,
 51,
 -60,
 -59,
 -7,
 -2,
 36,
 -23,
 -42,
 1,
 -48,
 -71,
 -34,
 -17,
 23,
 64,
 58,
 -28,
 -48,
 5,
 41,
 28,
 63,
 86,
 118,
 28,
 -43,
 -47,
 16,
 116,
 99,
 1,
 -26,
 41,
 56,
 144,
 11,
 8,
 -11,
 24,
 8,
 26,
 77,
 56,
 -19,
 31,
 96,
 182,
 87,
 32,
 40,
 72,
 51,
 -23,
 53,
 116,
 136,
 82,
 139,
 131,
 143,
 104,
 107,
 72,
 57,
 96,
 104,
 59,
 -15,
 -12,
 26,
 99,
 131,
 72,
 70,
 76,
 97,
 86,
 91,
 128,
 114,
 86,
 88,
 134,
 154,
 163,
 157,
 157,
 83,
 79,
 76,
 94,
 106,
 146,
 133,
 128,
 104,
 115,
 116,
 82,
 104,
 129,
 136,
 121,
 124,
 120,
 172,
 148,
 100,
 104,
 135,
 154,
 161,
 171,
 160,
 179,
 188,
 204,
 197,
 178,
 155,
 154,
 212,
 278,
 210,
 174,
 162,
 164,
 202,
 204,
 207,
 233,
 283,
 178,
 203,
 195,
 219,
 200,
 222,
 218,
 237,
 244,
 218,
 218,
 205,
 242,
 239,
 233,
 203,
 252,
 217,
 173,
 205,
 221,
 191,
 192,
 219,
 258,
 270,
 243,
 22

## Deriving Date Column for Merging

In [15]:
# Creating a data frame to store results
df_temp = pd.DataFrame()

# Creating date column with date time format
df_temp['date'] = [datetime.strptime(dte, '%Y-%m-%dT%H:%M:%S') for dte in dates_temp]

# Converting temperatures from tenths of Celsius to normal Celsius
df_temp['avgTemp'] = [float(v)/10.0 for v in temps]

df_temp.tail()

,date,avgTemp
360,2022-12-27,-0.7
361,2022-12-28,3.4
362,2022-12-29,6.4
363,2022-12-30,9.3
364,2022-12-31,8.2


# Combining Trip and Temperature Data

In [16]:
# Merging data frames
df_merged = df_trips.merge(df_temp, how='left', on='date', indicator='merged')
df_merged.tail()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date,avgTemp,merged
895480,D438F1622839AC50,classic_bike,2022-12-06 15:43:38,2022-12-06 15:53:57,Dey St,JC065,Riverview Park,JC057,40.737828,-74.067083,40.744319,-74.043991,member,2022-12-06,9.3,both
895481,747A63A8E782D171,electric_bike,2022-12-08 08:17:51,2022-12-08 08:23:33,9 St HBLR - Jackson St & 8 St,HB305,City Hall - Washington St & 1 St,HB105,40.747907,-74.038412,40.737360,-74.030970,casual,2022-12-08,11.6,both
895482,AE090858CFDE6E82,electric_bike,2022-12-23 14:10:07,2022-12-23 14:14:18,Mama Johnson Field - 4 St & Jackson St,HB404,City Hall - Washington St & 1 St,HB105,40.743140,-74.040041,40.737360,-74.030970,member,2022-12-23,7.5,both
895483,B3CC8E70AF4E259C,classic_bike,2022-12-02 04:43:25,2022-12-02 04:46:55,Mama Johnson Field - 4 St & Jackson St,HB404,City Hall - Washington St & 1 St,HB105,40.743135,-74.040080,40.737360,-74.030970,member,2022-12-02,3.4,both
895484,176B601F21327350,classic_bike,2022-12-30 14:50:17,2022-12-30 14:55:37,14 St Ferry - 14 St & Shipyard Ln,HB202,City Hall - Washington St & 1 St,HB105,40.752747,-74.024035,40.737360,-74.030970,member,2022-12-30,9.3,both


I proceeded to merge on the shared date column using a left merge to keep every record of the trip data while adding the corresponding daily temperature reading. The below "both" value count of 895485 confirms all rows were merged as expected, with no missing values in either data frames. This merged data frame will now facilitate the analysis of trip sensitivity to weather.

In [17]:
df_merged['merged'].value_counts(dropna=False)

merged
both          895485
left_only          0
right_only         0
Name: count, dtype: int64

# Saving Merged DataFrame

In [18]:
df_merged.to_csv(os.path.join(Path, 'Prepared Data', 'Citi_Bike_2022.csv'), index=False)